TÄSSÄ

In [93]:
#importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.utils import to_categorical

# Dataset Preparation

### Import dataset from the web

In [94]:
# prepare dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
dataset = pd.read_csv(url, names=names)
print(dataset.head())

   sepal-length  sepal-width  petal-length  petal-width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa


### Pre-preparation

In [95]:
# shuffle the dataset (splitting the dataset shuffles it later anyway, but here goes)
# I do not use np.random.shuffle(), because then I would have to convert the dataset to a numpy array
# and then back to a pandas dataframe, which is not very efficient
dataset = dataset.sample(frac=1).reset_index(drop=True)

# one-hot encode the class labels
dataset['class'] = pd.Categorical(dataset['class']).codes

# split the dataset into input and output features
X = pd.DataFrame(dataset.drop('class', axis=1))
Y = to_categorical(dataset['class'])

print(X.head())
print(Y[:5])


   sepal-length  sepal-width  petal-length  petal-width
0           6.3          2.5           4.9          1.5
1           6.9          3.1           5.4          2.1
2           5.5          2.4           3.8          1.1
3           7.3          2.9           6.3          1.8
4           6.0          3.4           4.5          1.6
[[0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


# höpö höpö

In [105]:
# fit
model = Sequential([
    Input(shape=(4,)),
    Dense(48, activation='relu'),
    Dense(3, activation='softmax')
])



model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

model.fit(X, Y, epochs=5)

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_20 (Dense)                │ (None, 48)             │           240 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 3)              │           147 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 387 (1.51 KB)

 Trainable params: 387 (1.51 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.3477 - loss: 1.9647 
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3181 - loss: 1.5980 
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.3428 - loss: 1.4257
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.3201 - loss: 1.3387 
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.0525 - loss: 1.2723
